<a href="https://colab.research.google.com/github/Guidevit/notebooks/blob/main/querypipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_openai llama-index llama_hub tavily-python arize_phoenix

In [ ]:
!pip install numpy==1.23.5

In [ ]:
!pip install tqdm

In [ ]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index

px.launch_app()
llama_index.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit https://mqeok2l74mp2-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
from langchain_openai import ChatOpenAI
import openai

openai_api_key = " "
openai.api_key = openai_api_key


llm = ChatOpenAI(model="gpt-3.5-turbo",
                 temperature=0,
                 openai_api_key=openai_api_key,
                 max_tokens=250
                 )

In [ ]:
import os
os.environ["TAVILY_API_KEY"] = " "
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
import json
from llama_index import VectorStoreIndex
from llama_index import download_loader
from llama_index.indices.query.query_transform.base import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import (
    TransformQueryEngine,
)
from IPython.display import Markdown, display
from langchain_community.tools.tavily_search import TavilySearchResults
from llama_index.prompts import PromptTemplate
from llama_index.query_pipeline.query import QueryPipeline
from llama_index.postprocessor import CohereRerank
from llama_index.response_synthesizers import TreeSummarize
from llama_index.query_pipeline.query import QueryPipeline
from llama_index import ServiceContext
from llama_index.query_pipeline import CustomQueryComponent
from typing import Dict, Any
import json
from llama_index import VectorStoreIndex, download_loader
from IPython.display import Markdown, display
from llama_hub.tools.tavily_research import TavilyToolSpec
from llama_index.agent import OpenAIAgent
from requests.exceptions import HTTPError
from llama_index import Document



# ABORDAGEM DE DECOMPOSIÇÃO DE RACIOCÍNIO
Para abordar a questão como um jurista especializado em todas as matérias do
direito brasileiro e decompor perguntas complexas em questões mais específicas,
pode-se adotar uma técnica sistemática e analítica. Essa técnica envolve várias
etapas:

## Inicialização e Preparação:

* Defina a original_query_string com a pergunta do aluno.
* Inicialize uma instância de TavilySearchResults ou um equivalente para
realizar pesquisas na web.[link text](https://)

In [ ]:
query_str = """Considerando a complexidade e a diversidade das fontes
do Direito Civil brasileiro, como os princípios gerais do direito, a
legislação, a jurisprudência e os costumes interagem e influenciam a
interpretação e a aplicação do Código Civil, em particular em situações
que envolvem conflitos de normas? Além disso, como a doutrina e a
analogia desempenham seus papéis na resolução de lacunas legais, e
de que maneira as decisões judiciais recentes têm refletido a
dinâmica entre essas fontes, especialmente em casos que tratam de
direitos da personalidade e obrigações contratuais?"""



## Etapas do Processo:

Para cada etapa, você fará uma busca usando o TavilySearchResults, analisará os resultados, e os utilizará para refinar sua compreensão e resposta à pergunta original. O processo para cada etapa seria:

### a. Identificação de Palavras-Chave (IDENT_KEY_WORDS):

Busque as palavras-chave identificadas e entenda como elas se relacionam com a questão legal em foco.

### c. Estruturação Lógica (IDENT_LOGIC):

Com base nos resultados das buscas anteriores, estruture a questão em subcomponentes lógicos.

In [ ]:
class LogicalStructureIdentificationComponent(CustomQueryComponent):
    def _validate_component_inputs(self, input: Dict[str, Any]) -> Dict[str, Any]:
        """Validates the inputs of the component during run_component."""
        required_keys = {'query_str'}
        missing_keys = required_keys - input.keys()
        if missing_keys:
            raise ValueError(f"Missing required input(s) for LogicalStructureIdentificationComponent: {', '.join(missing_keys)}")
        return input

    @property
    def _input_keys(self) -> set:
        """Input keys of the component."""
        return {"query_str"}

    @property
    def _output_keys(self) -> set:
        """Output keys of the component."""
        return {"logical_structure", "query_str"}

    def _run_component(self, **kwargs) -> Dict[str, Any]:
        llm = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=0.5,
                  openai_api_key=openai_api_key,
                  max_tokens=4096
                  )
        query_str = kwargs['query_str']

        # Combine search results texts to provide as context
        ident_logic_str = f"""Com base na pergunta original "{query_str}",
        identifique e estruture a questão em componentes lógicos.

        Agora, analise e divida a questão em subcomponentes lógicos. Identifique
        os principais conceitos jurídicos e questões subordinadas, e esclareça
        as relações entre eles que são fundamentais para uma compreensão
        abrangente do tema. Considere os seguintes aspectos na sua estruturação lógica:

        1. Quais são os conceitos jurídicos chave abordados na pergunta?
        2. Existem subquestões que precisam ser individualmente analisadas?
        3. Como esses conceitos e subquestões se inter-relacionam dentro do contexto
        ?"""

        # Assuming you have a way to load and index your documents, similar to the keyword identification
        StringIterableReader = download_loader("StringIterableReader")
        loader = StringIterableReader()
        documents = loader.load_data(texts=[ident_logic_str])
        index = VectorStoreIndex(documents)

        ident_logic_tmpl = PromptTemplate(ident_logic_str)
        prompt = ident_logic_tmpl.format(query_str=query_str)

        # Use the index as a query engine
        query_engine = index.as_query_engine()

        # Query the engine with the prompt
        logical_structure = query_engine.query(prompt)

        return {"logical_structure": logical_structure, "query_str": query_str}


### d. Questionamento Socrático (IDENT_QUEST_SOC):

Faça perguntas detalhadas para aprofundar sua compreensão de cada aspecto da questão, utilizando as informações coletadas anteriormente.

In [ ]:
from tqdm import tqdm
from tqdm.auto import tqdm
from typing import List, Dict, Any
from pydantic import BaseModel
from llama_index import VectorStoreIndex, ServiceContext, PromptTemplate
from llama_index.llms import OpenAI
from llama_index.agent import OpenAIAgent
from llama_hub.tools.tavily_research import TavilyToolSpec
from nest_asyncio import apply
import asyncio

apply()

class SocraticQuestionOutput(BaseModel):
    """Data model for the output of SocraticQuestioningComponent."""
    socratic_questions: List[str]

class SocraticQuestioningComponent(CustomQueryComponent):
    def _validate_component_inputs(self, input: Dict[str, Any]) -> Dict[str, Any]:
        """Validates the inputs of the component during run_component."""
        required_keys = {'query_str', "logical_structure"}
        missing_keys = required_keys - input.keys()
        if missing_keys:
            raise ValueError(f"Missing required input(s) for SocraticQuestioningComponent: {', '.join(missing_keys)}")
        return input

    @property
    def _input_keys(self) -> set:
        """Input keys of the component."""
        return {"query_str", "logical_structure"}

    @property
    def _output_keys(self) -> set:
        """Output keys of the component."""
        return {"socratic_questions", "all_search_results", "all_sources"}

    def search_tavily(self, question, agent):
        print(f"Searching for question: {question}")  # Debug: Log the question being searched
        response_obj = agent.chat(question)  # Make synchronous call
        search_result = response_obj.response  # Access the response attribute
        # Use getattr to handle missing 'sources' attribute and debug print
        sources = getattr(response_obj, 'sources', [])
        print(f"Search Result: {search_result}")  # Debug: Log the raw search result
        print(f"Sources for question '{question}': {sources}")  # Debug: Log the sources
        return search_result, sources

    def _run_component_sync(self, **kwargs) -> Dict[str, Any]:
        llm = ChatOpenAI(model="gpt-3.5-turbo-1106",
            temperature=0,
            openai_api_key=openai_api_key,
            max_tokens=9000
            )
        query_str = kwargs['query_str']
        logical_structure = kwargs['logical_structure']  # Ensure this line is correct

        # Construct the ident_quest_soc_str here with proper formatting
        ident_quest_soc_str = f"""
        Baseado na pergunta original e na decomposição lógica identificado,
        formule perguntas socráticas que explorem a questão mais profundamente.
        Aqui está a pergunta ou questão levantada: {query_str}
        Aqui está a estrutura lógica da questão: {logical_structure}

        Formule perguntas socráticas que ajudem a explorar a questão em profundidade, questionando suposições, clarificando conceitos e revelando conexões e distinções críticas.
        Perguntas socráticas propostas:
        """

        # Assuming you have a way to load and index your documents, similar to the previous components
        StringIterableReader = download_loader("StringIterableReader")
        loader = StringIterableReader()
        documents = loader.load_data(texts=[ident_quest_soc_str])
        index = VectorStoreIndex(documents)

        ident_quest_soc_tmpl = PromptTemplate(ident_quest_soc_str)
        prompt = ident_quest_soc_tmpl.format(query_str=query_str)

        # Use the index as a query engine
        query_engine = index.as_query_engine()

        # Use the index as a query engine with appropriate response mode
        query_engine = index.as_query_engine(
            output_cls=SocraticQuestionOutput,  # Use your Pydantic model here
            response_mode="tree_summarize"      # Choose an appropriate response mode
        )

        # Query the engine with the prompt
        socratic_questions = query_engine.query(prompt)

        # Check the structure of socratic_questions
        if not hasattr(socratic_questions, 'socratic_questions') or not socratic_questions.socratic_questions:
            raise ValueError("The response structure is unexpected or socratic_questions are missing.")

        questions = socratic_questions.socratic_questions
        print(f"Socratic Questions: {questions}")  # Debug: Log all socratic questions

        # Initialize Tavily tool
        tavily_tool = TavilyToolSpec()
        agent = OpenAIAgent.from_tools(tavily_tool.to_tool_list())

        # Process each question synchronously
        all_search_results = []
        # Use a dictionary to store sources for each question
        all_sources = {}
        for question in questions:
            result, sources = self.search_tavily(question, agent)
            all_search_results.append(result)
            # Store sources in the dictionary with the question as the key
            all_sources[question] = sources
            print(f"Result for question '{question}': {result}")
            print(f"Sources for question '{question}': {all_sources[question]}")

        return {
            "socratic_questions": socratic_questions,
            "all_search_results": all_search_results,
            "all_sources": all_sources  # Include the collected sources
        }

    def _run_component(self, **kwargs) -> Dict[str, Any]:
        """Execute the component synchronously."""
        return self._run_component_sync(**kwargs)


### e. Checagem de Consistência e Completude (IDENT_CONS_COMPL):

Verifique se todas as partes da questão foram abordadas, buscando informações adicionais se necessário.

### f. Identificação de Referências e Fontes (IDENT_REF_FONTES):

Busque leis, regulamentos, decisões judiciais ou literatura acadêmica relevantes para a questão.

### g. Síntese e Conclusão (SINT_CONCL):

Sintetize todas as informações coletadas para construir uma resposta coerente e bem fundamentada.

### h. Revisão Crítica (REV_CRIT):

Revise criticamente a resposta elaborada para garantir sua precisão, clareza e compreensibilidade.

## Automação e Iteração:

Para cada etapa, você pode automatizar a busca e análise de informações utilizando scripts ou funções específicas. Por exemplo, você pode criar funções que recebam a pergunta do aluno e retornem os resultados das pesquisas, bem como análises desses resultados.

## Integração dos Resultados:

* Ao final de cada etapa, integre os resultados obtidos na construção da resposta final.
* Use os insights de cada etapa para refinar e aprofundar a resposta, garantindo que todos os aspectos da questão sejam adequadamente abordados.

## Geração de Documentação:

Documente cada etapa do processo, incluindo as perguntas feitas, as buscas realizadas e os resultados obtidos.
Isso não apenas ajuda na construção da resposta final, mas também serve como um registro detalhado do processo de pesquisa e análise.

# CONFIGURAÇÃO DO **PIPELINE**

In [ ]:
# Configure the query pipeline
p = QueryPipeline(verbose=True)

# Add modules to the pipeline
p.add_modules(
    {
        "logical_structure": LogicalStructureIdentificationComponent(),  # Existing component
        "socratic_questioning": SocraticQuestioningComponent()  # New component
    }
)

# Link logical_structure to socratic_questioning
p.add_link("logical_structure", "socratic_questioning", src_key="logical_structure", dest_key="logical_structure")
p.add_link("logical_structure", "socratic_questioning", src_key="query_str", dest_key="query_str")


# Prepare the input for the pipeline
input_dict = {
    "logical_structure": {"query_str": query_str}
}

# Run the pipeline
output_dict = p.run_multi(input_dict)

# The output_dict will now include the output from SocraticQuestioningComponent as well.


> Running module logical_structure with input: 
query_str: Considerando a complexidade e a diversidade das fontes
do Direito Civil brasileiro, como os princípios gerais do direito, a
legislação, a jurisprudência e os costumes interagem e influenciam a
interpr...

> Running module socratic_questioning with input: 
logical_structure: 1. Os conceitos jurídicos chave abordados na pergunta são: princípios gerais do direito, legislação, jurisprudência, costumes, interpretação e aplicação do Código Civil, conflitos de normas, doutrina,...
query_str: Considerando a complexidade e a diversidade das fontes
do Direito Civil brasileiro, como os princípios gerais do direito, a
legislação, a jurisprudência e os costumes interagem e influenciam a
interpr...

Socratic Questions: ['Como os princípios gerais do direito, a legislação, a jurisprudência e os costumes interagem e influenciam a interpretação e a aplicação do Código Civil em situações que envolvem conflitos de normas?', 'Como a doutrina e a a

In [ ]:
output_dict

{'socratic_questioning': {'socratic_questions': PydanticResponse(response=SocraticQuestionOutput(socratic_questions=['Como os princípios gerais do direito, a legislação, a jurisprudência e os costumes interagem e influenciam a interpretação e a aplicação do Código Civil em situações que envolvem conflitos de normas?', 'Como a doutrina e a analogia desempenham seus papéis na resolução de lacunas legais?', 'De que maneira as decisões judiciais recentes têm refletido a dinâmica entre essas fontes, especialmente em casos que tratam de direitos da personalidade e obrigações contratuais?']), source_nodes=[NodeWithScore(node=Document(id_='0f3182e8-e628-46d2-8c4a-824618558445', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n        Baseado na pergunta original e na decomposição lógica identificado,\n        formule perguntas socráticas que explorem a questão mais profundamente.\n        Aqui está a pergunta ou questão leva

In [ ]:
# Etapa de Estruturação Lógica (IDENT_LOGIC)
IDENT_LOGIC_str = f"""
Com base nos resultados das buscas anteriores, estruture a questão em subcomponentes lógicos.
Aqui está a pergunta ou questão levantada: {original_query_string}
"""
IDENT_LOGIC_tmpl = PromptTemplate(IDENT_LOGIC_str)
p_ident_logic = QueryPipeline(chain=[IDENT_LOGIC_tmpl, llm], verbose=True)
output_ident_logic = p_ident_logic.run(query_str=original_query_string)
display(Markdown(f"<b>{output_ident_logic}</b>"))

# Etapa de Questionamento Socrático (IDENT_QUEST_SOC)
IDENT_QUEST_SOC_str = f"""
Faça perguntas detalhadas para aprofundar sua compreensão de cada aspecto da questão, utilizando as informações coletadas anteriormente.
Aqui está a pergunta ou questão levantada: {original_query_string}
"""
IDENT_QUEST_SOC_tmpl = PromptTemplate(IDENT_QUEST_SOC_str)
p_ident_quest_soc = QueryPipeline(chain=[IDENT_QUEST_SOC_tmpl, llm], verbose=True)
output_ident_quest_soc = p_ident_quest_soc.run(query_str=original_query_string)
display(Markdown(f"<b>{output_ident_quest_soc}</b>"))

# Etapa de Checagem de Consistência e Completude (IDENT_CONS_COMPL)
IDENT_CONS_COMPL_str = f"""
Verifique se todas as partes da questão foram abordadas, buscando informações adicionais se necessário.
Aqui está a pergunta ou questão levantada: {original_query_string}
"""
IDENT_CONS_COMPL_tmpl = PromptTemplate(IDENT_CONS_COMPL_str)
p_ident_cons_compl = QueryPipeline(chain=[IDENT_CONS_COMPL_tmpl, llm], verbose=True)
output_ident_cons_compl = p_ident_cons_compl.run(query_str=original_query_string)
display(Markdown(f"<b>{output_ident_cons_compl}</b>"))

# Etapa de Identificação de Referências e Fontes (IDENT_REF_FONTES)
IDENT_REF_FONTES_str = f"""
Busque leis, regulamentos, decisões judiciais ou literatura acadêmica relevantes para a questão.
Aqui está a pergunta ou questão levantada: {original_query_string}
"""
IDENT_REF_FONTES_tmpl = PromptTemplate(IDENT_REF_FONTES_str)
p_ident_ref_fontes = QueryPipeline(chain=[IDENT_REF_FONTES_tmpl, llm], verbose=True)
output_ident_ref_fontes = p_ident_ref_fontes.run(query_str=original_query_string)
display(Markdown(f"<b>{output_ident_ref_fontes}</b>"))

# Etapa de Síntese e Conclusão (SINT_CONCL)
SINT_CONCL_str = f"""
Sintetize todas as informações coletadas para construir uma resposta coerente e bem fundamentada.
Aqui está a pergunta ou questão levantada: {original_query_string}
"""
SINT_CONCL_tmpl = PromptTemplate(SINT_CONCL_str)
p_sint_concl = QueryPipeline(chain=[SINT_CONCL_tmpl, llm], verbose=True)
output_sint_concl = p_sint_concl.run(query_str=original_query_string)
display(Markdown(f"<b>{output_sint_concl}</b>"))

# Etapa de Revisão Crítica (REV_CRIT)
REV_CRIT_str = f"""
Revise criticamente a resposta elaborada para garantir sua precisão, clareza e compreensibilidade.
Aqui está a pergunta ou questão levantada: {original_query_string}
"""
REV_CRIT_tmpl = PromptTemplate(REV_CRIT_str)
p_rev_crit = QueryPipeline(chain=[REV_CRIT_tmpl, llm], verbose=True)
output_rev_crit = p_rev_crit.run(query_str=original_query_string)
display(Markdown(f"<b>{output_rev_crit}</b>"))

NameError: name 'original_query_string' is not defined